# Tools to Bind to Python
## Henry Schreiner
## PyHEP 2018


# Focus

* What Python bindings do
* What tools are available
* How Python bindings work

# Caviats

* Will cover C++ and C binding only
* Will not cover every tool available
* Will not cover `cppyy` in detail (but see Enric's talk)
* Python 2 is dying, long live Python 3!
    * but this talk is Py2 compatible also

# Selection:
* C Binding: ctypes, CFFI
* Manual binding (quick look)
* SWIG (Too automatic)
* Cython (Two languages)
* PyBind11 (just right)
* Special mention: Numba, cppyy

# Preperation

This talk is interactive, and can be run in SWAN:

[![Open in SWAN](http://swanserver.web.cern.ch/swanserver/images/badge_swan_white_150.png)](https://cern.ch/swanserver/cgi-bin/go?projurl=https://github.com/henryiii/pybindings_cc.git)

If you want to run it manually, just download the repository from github: [henryiii/pybindings_cc](https://github.com/henryiii/pybindings_cc)


Since this talk is an interactive notebook, *no code will be hidden* from you. Here are the required packages:

In [39]:
!pip install --user cffi pybind11 numba
# Other requirements: cython cppyy (SWIG is also needed but not a python module)
# Using Anaconda recommended for users not using SWAN

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Other requirements:
* cython, cppyy
* SWIG is also needed but not a python module
* Using Anaconda recommended for users not using SWAN

And, here are the standard imports. We will also add two variables to help with compiling:

In [41]:
from __future__ import print_function
import os
import sys
from pybind11 import get_include
inc = '-I ' + get_include(user=True) + ' -I ' + get_include(user=False)
plat = '-undefined dynamic_lookup' if 'darwin' in sys.platform else '-fPIC'
print('inc:', inc)
print('plat:', plat)

inc: -I /eos/user/h/hschrein/.local/include/python3.6m -I /cvmfs/sft-nightlies.cern.ch/lcg/nightlies/dev3python3/Mon/Python/3.6.5/x86_64-slc6-gcc62-opt/include/python3.6m
plat: -fPIC


# What is meant by bindings?

Bindings allow a function(alitiy) in a library to be accessed from Python.

#### We will start with this example:

In [38]:
%%writefile simple.c

float square(float x) {
    return x*x;
}

Overwriting simple.c


#### Desired usage in Python:
```python
y = square(x)
```

C bindings are very easy. Just compile into a shared library, then open it in python with the built in [ctypes](https://docs.python.org/3.7/library/ctypes.html) module:

In [4]:
!cc simple.c -shared -o simple.so

In [42]:
from ctypes import cdll, c_float
lib = cdll.LoadLibrary('./simple.so')
lib.square.argtypes = (c_float,)
lib.square.restype = c_float
lib.square(2.0)

4.0

* This may be all you need! See [AmpGen](https://gitlab.cern.ch/lhcb/Gauss/blob/LHCBGAUSS-1058.AmpGenDev/Gen/AmpGen/options/ampgen.py) Python interface.
* In [Pythonista](http://omz-software.com/pythonista/) for iOS, we can even use ctypes to access Apple's public APIs!

### [CFFI](http://cffi.readthedocs.io/en/latest/overview.html)

* The *C Foreign Function Interface* for Python
* Still C only
* Developed for PyPy, but available in CPython too

The same example as before:

In [43]:
from cffi import FFI
ffi = FFI()
ffi.cdef("float square(float);")
C = ffi.dlopen('./simple.so')
C.square(2.0)

4.0

# CPython

* Let's see how bindings work before going into C++ binding tools
* This is how CPython itself is implemented

> C reminder: `static` means visible in this file only

In [8]:
%%writefile pysimple.c
#include <Python.h>

float square(float x) {return x*x; }

static PyObject* square_wrapper(PyObject* self, PyObject* args) {
  float input, result;
  if (!PyArg_ParseTuple(args, "f", &input)) {return NULL;}
  result = square(input);
  return PyFloat_FromDouble(result);}

static PyMethodDef pysimple_methods[] = {
 { "square", square_wrapper, METH_VARARGS, "Square function" },
 { NULL, NULL, 0, NULL } };

#if PY_MAJOR_VERSION >= 3
static struct PyModuleDef pysimple_module = {
    PyModuleDef_HEAD_INIT, "pysimple", NULL, -1, pysimple_methods};
PyMODINIT_FUNC PyInit_pysimple(void) {
    return PyModule_Create(&pysimple_module); }
#else
DL_EXPORT(void) initpysimple(void) {
  Py_InitModule("pysimple", pysimple_methods); }
#endif

Overwriting pysimple.c


### Build:

In [9]:
!cc {inc} -shared -o pysimple.so pysimple.c {plat}

### Run:

In [45]:
import pysimple
pysimple.square(2.0)

4.0

# C++: Why do we need more?

* Sometimes simple is enough!
* `export "C"` allows C++ backend
* C++ API can have: overloading, classes, memory managment, etc...
* We could manually translate everything using C API

### Solution:

C++ binding tools!

This is our C++ example:

In [11]:
%%writefile SimpleClass.hpp
#pragma once

class Simple {
    int x;
  public:
    Simple(int x): x(x) {}
    int get() const {return x;}
};

Overwriting SimpleClass.hpp


# SWIG : automated bindings

* Produces "automatic" bindings
* Works with many output languages
* Has supporting module built into CMake

Downsides:

* Can be all or nothing
* Hard to customize
* Customizations tend to be language specific
* Slow development

In [12]:
%%writefile SimpleSWIG.i

%module simpleswig
%{
/* Includes the header in the wrapper code */
#include "SimpleClass.hpp"
%}
 
/* Parse the header file to generate wrappers */
%include "SimpleClass.hpp"

Overwriting SimpleSWIG.i


In [13]:
!swig -swiglib

/build/jenkins/workspace/install/swig/3.0.12/x86_64-slc6-gcc62-opt/share/swig/3.0.12


#### SWAN/LxPlus only:

We need to fix the `SWIG_LIB` path if we are using LCG's version of SWIG (such as on SWAN)

In [14]:
if 'LCG_VIEW' in os.environ:
    swiglibold = !swig -swiglib
    swigloc = swiglibold[0].split('/')[-3:]
    swiglib = os.path.join(os.environ['LCG_VIEW'], *swigloc)
    os.environ['SWIG_LIB'] = swiglib

In [15]:
!swig -python -c++ SimpleSWIG.i

In [16]:
!c++ -shared SimpleSWIG_wrap.cxx {inc} -o _simpleswig.so {plat}

In [17]:
import simpleswig
x = simpleswig.Simple(2)
x.get()

2

# Cython

* Built to be a Python+C language for high performance compututations
* Performance computation space in competition with numba
* Due to design, also makes binding easy
* Easy to customize result
* Can write Python 2 or 3, regardless of calling language

Downsides:
* Requires learning a new(ish) language
* Have to think with three hats
* *Very* verbose

# Speed comparison

In [18]:
def f(x):
    for _ in range(100000000):
        x=x+1
    return x

In [19]:
%%timeit
f(1)

4.63 s ± 27.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%load_ext Cython

In [21]:
%%cython

def f(int x):
    for _ in range(10000000):
        x=x+1
    return x

In [22]:
%%timeit
f(23)

68.6 ns ± 6.83 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [24]:
import numba
@numba.jit
def f(x):
    for _ in range(10000000):
        x=x+1
    return x

In [25]:
%time
f(41)

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 23.8 µs


10000041

In [26]:
%%timeit
f(41)

216 ns ± 23.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


# Binding with Cython

In [27]:
%%writefile simpleclass.pxd
# distutils: language = c++

cdef extern from "SimpleClass.hpp":
    cdef cppclass Simple:
        Simple(int x)
        int get()

Overwriting simpleclass.pxd


In [28]:
%%writefile cythonclass.pyx
# distutils: language = c++

from simpleclass cimport Simple as cSimple

cdef class Simple:
    cdef cSimple *cself
    
    def __cinit__(self, int x):
        self.cself = new cSimple(x)
    
    def get(self):
        return self.cself.get()
    
    def __dealloc__(self):
        del self.cself

Overwriting cythonclass.pyx


In [29]:
!cythonize cythonclass.pyx

Compiling /eos/user/h/hschrein/SWAN_projects/pybindings_cc/cythonclass.pyx because it changed.
[1/1] Cythonizing /eos/user/h/hschrein/SWAN_projects/pybindings_cc/cythonclass.pyx


In [30]:
!g++ cythonclass.cpp -shared {inc} -o cythonclass.so {plat}

In [31]:
import cythonclass
x = cythonclass.Simple(3)
x.get()

3

# PyBind11

* Designed for making bindings
* Pure C++11 (no new language required)
* Builds remain simple and don't require preprocessing
* Easy to customize result
* Great Gitter community
* Similar to Boost::Python, but easier to build

Downsides:
* Still verbose
* Development variable
* Requires C++11 (and better with C++14)

In [32]:
%%writefile pybindclass.cpp

#include <pybind11/pybind11.h>
#include "SimpleClass.hpp"

namespace py = pybind11;

PYBIND11_MODULE(pybindclass, m) {
    py::class_<Simple>(m, "Simple")
        .def(py::init<int>())
        .def("get", &Simple::get)
    ;
}

Overwriting pybindclass.cpp


In [33]:
!c++ -std=c++11 pybindclass.cpp -shared {inc} -o pybindclass.so {plat}

In [34]:
import pybindclass
x = pybindclass.Simple(4)
x.get()

4

# Cppyy

* Born from ROOT bindings
* Built on top of Cling
* JIT, so can handle templates

Downsides:

* Header code runs in Cling
* Can be slow (PyPy supposed to be faster)
* ROOT vs. PyPy vs. pip versions
* Harder to customize result than PyBind11
* Broken on SWAN (so will not show working example here)

In [35]:
import cppyy

In [36]:
cppyy.include('SimpleClass.hpp')

AttributeError: module 'cppyy' has no attribute 'include'

In [ ]:
x = cppyy.gbl.Simple(5)
x.get()